In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tensorflow-addons

import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pickle
import numpy as np
import urllib3
import shutil
import zipfile
import itertools
from threading import Thread
import random

### Download File

In [ ]:
ALL             = 0
WORD_LEVEL      = 1
ENCODER_DECODER = 2

colab_base = '/content/drive/MyDrive/ashraful/'
pc_base    = './'
base       = colab_base


# dataset_paths = [new_dataset_path, top_50k_word_file, top20k_path, top20k_path]
dataset_paths = [base + 'models/LSTM/test-dataset(1).txt']
test_dataset_path = base + 'models/LSTM/test-dataset.txt'

# char-level-model paths
checkpoint_dir            = base + 'models/LSTM/char-level'
model_weights_path        = base + 'models/LSTM/char-level/weights'
input_tokenizer_dir_char  = base + 'models/LSTM/char-level/input-tokenizer.pickle'
target_tokenizer_dir_char = base + 'models/LSTM/char-level/target-tokenizer.pickle'

# word-level-model paths
saved_model_dir_base      = base + 'models/LSTM/word-level-new/'
saved_model_word          = saved_model_dir_base + 'word-level.h5'
input_tokenizer_dir_word  = saved_model_dir_base + 'input-tokenizer.pickle'
target_tokenizer_dir_word = saved_model_dir_base + 'target-tokenizer.pickle'

### Create Dataset

In [ ]:
# Splitting dataset into train test
lines = list()

for path in dataset_paths:
    lines.extend(io.open(path, encoding='UTF-8').read().strip().split('\n'))

# lines = list(lines)
lines.sort()
print(len(lines))

# word_pairs = [[[char for char in '<' + w.replace('ঃ\n', '').replace('\n', '') + '>'] for w in l.split(',')] for l in lines]

# print(word_pairs[0][0])
# print(word_pairs[0][1])

# inp_lang, targ_lang = zip(*word_pairs)

# inp_lang_train, inp_lang_val, targ_lang_train, targ_lang_val = \
#             train_test_split(inp_lang, targ_lang, test_size=0.2, random_state=4651)



10000


In [ ]:
word_pairs = [[w for w in l.split(',')] for l in lines]

print(word_pairs[0][0])
print(word_pairs[0][1])

inp_lang, targ_lang = zip(*word_pairs)
inp_lang_train, inp_lang_val, targ_lang_train, targ_lang_val = \
            train_test_split(inp_lang, targ_lang, test_size=0.2, random_state=4651)

# inp_lang_train  = inp_lang_train.to_numpy()
# inp_lang_val    = inp_lang_val.to_numpy()
# targ_lang_train = targ_lang_train.to_numpy()
# targ_lang_val   = targ_lang_val.to_numpy()

abagprobon
আবেগপ্রবণ


### Char-Level Model Parameters 

In [ ]:
# Retrieving tokenizer for char-level
with open(input_tokenizer_dir_char, mode='rb') as data_file:
    inp_lang_tokenizer_char = pickle.load(data_file)
with open(target_tokenizer_dir_char, mode='rb') as data_file:
    tar_lang_tokenizer_char = pickle.load(data_file)

In [ ]:
vocab_inp_size = len(inp_lang_tokenizer_char.word_index) + 1
vocab_tar_size = len(tar_lang_tokenizer_char.word_index) + 1
max_length_input = 20
max_length_output = 20

print(inp_lang_tokenizer_char.word_index)
print(tar_lang_tokenizer_char.word_index)

embedding_dims = 32
rnn_units = 256
dense_units = 256
Dtype = tf.float32

Tx = 20
Ty = 20

print(vocab_inp_size, vocab_tar_size)

{'<': 1, '>': 2, 'a': 3, 'o': 4, 'e': 5, 'r': 6, 'i': 7, 'h': 8, 'n': 9, 't': 10, 's': 11, 'k': 12, 'u': 13, 'b': 14, 'l': 15, 'd': 16, 'm': 17, 'p': 18, 'c': 19, 'g': 20, 'j': 21, 'y': 22, 'w': 23, 'f': 24, 'v': 25, 'q': 26, 'z': 27, 'x': 28}
{'<': 1, '>': 2, 'া': 3, 'র': 4, 'ে': 5, 'ি': 6, '্': 7, 'ন': 8, 'ক': 9, 'স': 10, 'ব': 11, 'ল': 12, 'ম': 13, 'ত': 14, 'ু': 15, 'প': 16, 'ট': 17, 'দ': 18, 'ো': 19, 'জ': 20, 'গ': 21, 'ই': 22, 'হ': 23, 'শ': 24, 'ী': 25, 'য': 26, 'ড': 27, 'ভ': 28, 'য়': 29, 'ফ': 30, 'চ': 31, 'ও': 32, 'আ': 33, 'অ': 34, 'এ': 35, 'খ': 36, 'ষ': 37, 'ণ': 38, 'ং': 39, 'ধ': 40, 'থ': 41, 'উ': 42, 'ছ': 43, 'ূ': 44, 'ঁ': 45, 'ৃ': 46, 'ড়': 47, 'ঠ': 48, 'ঘ': 49, 'ঞ': 50, 'ঙ': 51, 'ৌ': 52, 'ৎ': 53, 'ৈ': 54, 'ঝ': 55, 'ঃ': 56, 'ঢ': 57, 'ঈ': 58, 'ঋ': 59, 'ঊ': 60, 'ঐ': 61, 'ঔ': 62, 'ঢ়': 63}
29 64


### Define Model

In [ ]:
BATCH_SIZE=64
class MyModel(tf.keras.Model):
    def __init__(self, input_vocab_size, output_vocab_size, embedding_dims, rnn_units):
        super().__init__()
        # Encoder
        self.input_vocab_size = input_vocab_size
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim=input_vocab_size,
                                                           output_dim=embedding_dims)
        self.encoder_rnnlayer1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(rnn_units, return_sequences=True))
        self.encoder_rnnlayer2 = tf.keras.layers.LSTM(rnn_units,
                                                      return_sequences=True,
                                                      return_state=True)
        self.encoder_norm = tf.keras.layers.BatchNormalization()

        # Decoder
        self.output_vocab_size = output_vocab_size
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim=output_vocab_size,
                                                           output_dim=embedding_dims) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, 
                                                sampler= self.sampler,
                                                output_layer=self.dense_layer)

        self.attention_mechanism.memory_initialized
        self.decoder_embedding_matrix = None


    def initialize_initial_state(self):
        self.initial_state = [
            tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, 
                                          memory = memory, 
                                          memory_sequence_length=memory_sequence_length)
        # return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decoder rnn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state
    
    def call(self, inputs, training=False):
        encoder_input, decoder_input = inputs

        x = self.encoder_embedding(encoder_input)
        x = self.encoder_rnnlayer1(x)
        x = self.encoder_norm(x, training=training)
        a, a_tx, c_tx = self.encoder_rnnlayer2(x)
        
        decoder_emb_inp = self.decoder_embedding(decoder_input)
        self.attention_mechanism.setup_memory(a)
        decoder_initial_state = self.build_decoder_initial_state(BATCH_SIZE,
                                                                encoder_state=[a_tx, c_tx],
                                                                Dtype=tf.float32)
        
        outputs, _, _ = self.decoder(decoder_emb_inp, 
                                     initial_state=decoder_initial_state,
                                     sequence_length=BATCH_SIZE*[Ty-1])

        return outputs
    
    def evaluate(self, inputs, beam_width=3):
        if self.decoder_embedding_matrix is None:
            self.decoder_embedding_matrix = tf.train.load_variable(
            model_weights_path, 'decoder_embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE')
            # print(self.decoder_embedding_matrix.shape)
        
        inference_batch_size = inputs.shape[0]
        # print(inputs.shape)
        result = ''

        x = self.encoder_embedding(inputs)
        # x = tf.one_hot(inputs, depth=self.input_vocab_size)
        x = self.encoder_rnnlayer1(x)
        x = self.encoder_norm(x, training=False)
        enc_out, enc_h, enc_c = self.encoder_rnnlayer2(x)

        dec_h = enc_h
        # dec_c = enc_c

        start_tokens = tf.fill([inference_batch_size], tar_lang_tokenizer_char.word_index['<'])
        # print(start_tokens)
        end_token = tar_lang_tokenizer_char.word_index['>']
        # print(end_token)

        enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
        self.attention_mechanism.setup_memory(enc_out)
        # print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

        # set decoder_inital_state which is an AttentionWrapperState considering beam_width
        hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=beam_width * inference_batch_size,
                                                                dtype=tf.float32)
        decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

        # Instantiate BeamSearchDecoder
        decoder_instance = tfa.seq2seq.BeamSearchDecoder(self.rnn_cell, 
                                                         beam_width=beam_width, 
                                                         output_layer=self.dense_layer)
        decoder_instance.maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)
        # decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0]

        # The BeamSearchDecoder object's call() function takes care of everything.
        outputs, final_state, sequence_lengths = decoder_instance(self.decoder_embedding_matrix, 
                                                                  start_tokens=start_tokens,
                                                                  end_token=end_token, 
                                                                  initial_state=decoder_initial_state)

        final_outputs = tf.transpose(outputs.predicted_ids, perm=(0, 2, 1))
        beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0, 2, 1))

        return final_outputs.numpy(), beam_scores.numpy()

model = MyModel(vocab_inp_size,vocab_tar_size, embedding_dims, rnn_units)
model.load_weights(filepath=model_weights_path)

### Char-level model Evaluation 

In [ ]:
def get_tensor(tokenizer, lang):
    tensor = tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                        padding='post',
                                                        maxlen=20, 
                                                        truncating='post')
    return tensor


input_tensor_train  = get_tensor(inp_lang_tokenizer_char, inp_lang_train)
input_tensor_val    = get_tensor(inp_lang_tokenizer_char, inp_lang_val)
target_tensor_train = get_tensor(tar_lang_tokenizer_char, targ_lang_train)
target_tensor_val   = get_tensor(tar_lang_tokenizer_char, targ_lang_val)


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = len(input_tensor_train)

train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

test_dataset  = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
test_dataset  = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Evaluate char-level train
def calculate_acc(dataset):
    beam_width = 10
    correct_count = np.array([0]*4)
    total_count = 0
    steps_per_epoch = len(dataset)
    print(steps_per_epoch)
    # exit(0)
    start = time.time()
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        # outputs, scores = beam_evaluate(inp, beam_width=beam_width)
        outputs, scores = model.evaluate(inp, beam_width=beam_width)
        # print(targ.shape)
        targ = list(map(get_bangla, targ.numpy()))
        targ = list(map(lambda x: x.replace('<', ''), targ))
        # print(targ)
        outputs = [list(map(get_bangla, output)) for output in outputs]
        # print(outputs)

        for i in range(len(targ)):
            if targ[i] == outputs[i][0]:
                correct_count[0]+=1
            if targ[i] in outputs[i][0:3]:
                correct_count[1]+=1
            if targ[i] in outputs[i][0:5]:
                correct_count[2]+=1
            if targ[i] in outputs[i]:
                correct_count[3]+=1
            total_count+=1

    print(f'Total size {total_count}')
    print(f'Acc@1 : {((correct_count[0]/total_count))*100:.2f} %')
    print(f'Acc@3 : {((correct_count[1]/total_count))*100:.2f} %')
    print(f'Acc@5 : {((correct_count[2]/total_count))*100:.2f} %')
    print(f'Acc@10: {((correct_count[3]/total_count))*100:.2f} %')
    print(f'Time taken: {(time.time() - start):.2f} s\n')

In [ ]:
calculate_acc(train_dataset)
calculate_acc(val_dataset)

### Shabdik Model Evaluation

In [ ]:
def get_bangla(array):
    bangla_list = list(map(lambda x: tar_lang_tokenizer_char.index_word[x] if x != 0 else '', array))
    bangla_list.append('>')
    return "".join(bangla_list[0:bangla_list.index('>')])


In [ ]:
def preprocess_char(word):
    word = [[char for char in ('<' + word.rstrip().lstrip() + '>')]]
    word = inp_lang_tokenizer_char.texts_to_sequences(word)
    inputs = tf.keras.preprocessing.sequence.pad_sequences(word, 
                                                           padding='post',
                                                           maxlen=20, 
                                                           truncating='post')
    # print(inputs)
    return tf.convert_to_tensor(inputs)

In [ ]:
def predict_for_char(english_word, result, k=10):
    # start = time.time()
    outputs, score = model.evaluate(preprocess_char(english_word), k)
    outputs = [list(map(get_bangla, output)) for output in outputs]
    result[0] = outputs[0]

    # print(f'Time taken: {(time.time() - start)*1000:.2f} ms\n')

In [ ]:
predictions = [None]
predict_for_char("ami", predictions)
print(predictions[0])

['আমি', 'অমি', 'এমই', 'আমী', 'আমই', 'এমি', 'অমী', 'এমী', 'মী', 'মি']


In [ ]:
# Loading word-level model
word_model = tf.keras.models.load_model(saved_model_word)
word_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 12, 32)            928       
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              164864    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 50001)             1650033   
                                                      

In [ ]:
# Retrieving tokenizer for word-level
with open(input_tokenizer_dir_word, mode='rb') as data_file:
    inp_lang_tokenizer_word = pickle.load(data_file)
with open(target_tokenizer_dir_word, mode='rb') as data_file:
    tar_lang_tokenizer_word = pickle.load(data_file)

In [ ]:
def preprocess_word(word):
    word = [[char for char in ('<' + word.rstrip().lstrip() + '>')]]
    word = inp_lang_tokenizer_word.texts_to_sequences(word)
    inputs = tf.keras.preprocessing.sequence.pad_sequences(word, 
                                                           padding='post',
                                                           maxlen=12, 
                                                           truncating='post')

    return tf.convert_to_tensor(inputs)

In [ ]:
def predict_for_word(input, result, k=10):
    if len(input) > 10:
        result[0] = []
        return

    predictions = tf.math.top_k(tf.reshape(word_model.predict(preprocess_word(input)), [-1]), k)
    values = predictions.values.numpy()
    indices = predictions.indices.numpy()

    for i in range(1, k):
        # print(values[i-1]/values[i])
        if values[i - 1] / values[i] > 5 or values[i] <= 0.20:
            result[0] = (list(map(lambda x: tar_lang_tokenizer_word.index_word[x], indices[0:i])), values[0:i])
            return
        

    result[0] = (list(map(lambda x: tar_lang_tokenizer_word.index_word[x], indices)), values)


In [ ]:
predictions = [None]
predict_for_word("kumir", predictions)
print(predictions[0])

(['কুমীর'], array([0.80083203], dtype=float32))


In [ ]:
# Fixed mapping for single character word
single_word_dic = {
    'a': ['আ', 'া', 'অ', 'এ', 'ে', 'আঃ', 'া' + 'ঁ', 'আ' + 'ঁ'],
    'b': ['ব', 'ভ', '্' + 'ব', 'ব' + 'ি'],
    'c': ['চ', 'ছ', 'ক', 'ঁ', '্' + 'চ', '্' + 'ছ', 'স' + 'ি'],
    'd': ['ড', 'দ', 'ধ', 'দঃ', 'ডঃ', 'ঢ', '্' + 'ধ', 'ড' + 'ি', 'দ' + 'ি'],
    'e': ['এ', 'ে', 'আ', 'া', 'ই', 'ি', 'ঈ', 'ী', '্' + 'য' + 'া'],
    'f': ['ফ', 'প', '্' + 'ফ', '্' + 'প', 'এ' + 'ফ'],
    'g': ['গ', 'ঘ', 'য', 'জ', 'ং', 'ঞ', '্', 'জ' + 'ি'],
    'h': ['হ', '্', 'এ' + 'ই' + 'চ'],
    'i': ['ই', 'ি', 'ঈ', 'ী', 'ৈ', 'আ' + 'ই'],
    'j': ['জ', 'য', 'ঝ', 'জ্ব', '্' + 'জ', '্' + 'য', '্' + 'ঝ', 'জ' + 'ে'],
    'k': ['ক', 'খ', 'ক্ষ', '্' + 'ক', '্' + 'খ', 'ক' + 'ে'],
    'l': ['ল', '্' + 'ল', 'এ' + 'ল'],
    'm': ['ম', '্' + 'ম', 'এ' + 'ম'],
    'n': ['ন', 'ণ', 'ঙ', 'ঞ', 'ং', '্' + 'ন', '্' + 'ণ', '্' + 'ঙ', 'এ' + 'ন'],
    'o': ['ও', 'ঐ', 'ো', 'য়', 'ঃ', 'ওঁ', 'ঔ', 'ৌ', 'ৈ'],
    'p': ['প', 'ফ', '্' + 'প', '্' + 'ফ', 'প' + 'ি'],
    'q': ['ক', '্' + 'ক', 'ক' + 'ি' + 'উ'],
    'r': ['র', 'ঢ়', 'ড়', 'ঋ', 'ৃ', '্' + 'র', 'র' + '্', 'আ' + 'র'],
    's': ['স', 'শ', 'ষ', 'সঃ', '্' + 'স', '্' + 'শ', '্' + 'ষ', 'এ' + 'স'],
    't': ['ট', 'ঠ', 'ত', 'থ', 'তঁ', 'ৎ', '্' + 'ট', '্' + 'ঠ', 'ট' + 'ি'],
    'u': ['উ', 'ু', 'ঊ', 'ূ', 'উঃ', 'উঁ', 'ই' + 'উ'],
    'v': ['ভ', 'ব', 'ভঁ', '্' + 'ভ', '্' + 'ব', 'ভ' + 'ি'],
    'w': ['ও', 'ৌ', 'ওঃ', 'ওঁ', 'ড' + 'া' + 'ব' + 'ল' + 'ু'],
    'x': ['ক্স', 'এক্স'],
    'y': ['য়', 'ইয়', 'ই', 'ি', 'ঈ', 'ী', 'ে', 'ও' + 'য়' + 'া' + 'ই'],
    'z': ['য', 'জ', 'ঝ', '্' + 'য', '্' + 'জ', '্' + 'ঝ', 'জ' + 'এ' + 'ড', 'জ' + 'ি']
}

for key in single_word_dic:
    for i in range(len(single_word_dic[key]), 9):
        single_word_dic[key].append('')



In [ ]:
def predict(english_word, option=ALL, k=10):
    if type(english_word) != type(''):
        return []
    
    english_word = english_word.lower()
 
    if len(english_word) == 0:
        return []

    # print(english_word)

    if len(english_word) == 1:
        return single_word_dic[english_word][0:k]

    word_result = [None]
    char_result = [None]

    if option == WORD_LEVEL:
        if len(english_word) > 10:
            return []
        predict_for_word(english_word, word_result, k)
        # print(word_result)
    elif option == ENCODER_DECODER or (option == ALL and len(english_word) > 10):
        predict_for_char(english_word, char_result, k)
    else:
        t1 = Thread(target=predict_for_char, args=(english_word, char_result, k, ))
        t1.start()
        t2 = Thread(target=predict_for_word, args=(english_word, word_result, k, ))
        t2.start()
        t2.join()
        t1.join()

    char_values = np.array([i*.1 for i in range(k)])
    char_values = char_values / np.sum(char_values)
    results = {}

    # print(word_result[0])
    # print(char_result[0])

    word_result = word_result[0]
    char_result = char_result[0]

    # print(char_result)
    # print(word_result)

    if word_result is not None:
        for i in range(len(word_result[1])):
            # if word_result[0][i] in results:
            results[word_result[0][i]] = word_result[1][i]

    if char_result is not None:
        for i in range(len(char_result)):
            if char_result[i] in results:
                results[char_result[i]] += char_values[i]
            else:
                results[char_result[i]]  = char_values[i]

    return [key for (key, value) in sorted(results.items(), key=lambda x: x[1], reverse=True)[:k]]


# Ignore

In [ ]:
char_results = np.array([0, 0, 0, 0])
word_results = np.array([0, 0, 0, 0])
combined_results = np.array([0, 0, 0, 0])

def update_result(true, prediction, results):
    if prediction is None or len(prediction) == 0:
        return results

    if true == prediction[0]:
        results[0]+=1
    if true in prediction[0:3]:
        results[1]+=1
    if true in prediction[0:5]:
        results[2]+=1
    if true in prediction:
        results[3]+=1
    return results


def predict(english_word, bangla_word, option=ALL, k=10):
    global char_results
    global word_results
    global combined_results

    if type(english_word) != type(''):
        return False
    
    english_word = english_word.lower()
 
    if len(english_word) == 0:
        return False

    # print(english_word)

    if len(english_word) == 1:
        return single_word_dic[english_word][0:k]

    word_result = [None]
    char_result = [None]

    if option == WORD_LEVEL:
        if len(english_word) > 10:
            return []
        predict_for_word(english_word, word_result, k)
        # print(word_result)
    elif option == ENCODER_DECODER or (option == ALL and len(english_word) > 10):
        predict_for_char(english_word, char_result, k)
    else:
        t1 = Thread(target=predict_for_char, args=(english_word, char_result, k, ))
        t1.start()
        t2 = Thread(target=predict_for_word, args=(english_word, word_result, k, ))
        t2.start()
        t2.join()
        t1.join()

    char_values = np.array([i*.1 for i in range(k)])
    char_values = char_values / np.sum(char_values)
    results = {}

    # print(word_result[0])
    # print(char_result[0])

    word_result = word_result[0]
    char_result = char_result[0]

    # print(char_result)
    # print(word_result)

    if word_result is not None:
        for i in range(len(word_result[1])):
            # if word_result[0][i] in results:
            results[word_result[0][i]] = word_result[1][i]

    if char_result is not None:
        for i in range(len(char_result)):
            if char_result[i] in results:
                results[char_result[i]] += char_values[i]
            else:
                results[char_result[i]]  = char_values[i]

    combined = [key for (key, value) in sorted(results.items(), key=lambda x: x[1], reverse=True)[:k]]

#     if bangla_word == combined[0] or \
#     (bangla_word in combined[0:3] or \
#         bangla_word in combined[0:5] or \
#    bangla_word in combined and random.random() > 0.90) or \
#         random.random() > 0.90:

            # (random.random() > .70 and bangla_word in combined[0:3]):
        # print(bangla_word, word_result)
        # word_results = update_result(bangla_word, word_result[0], word_results)
        # char_results = update_result(bangla_word, char_result, char_results)
    combined_results = update_result(bangla_word, combined, combined_results)
    return combined
    # elif bangla_word not in  char_result and word_result is not None and word_result[0] is not None:
    #     if bangla_word in word_result[0] or random.random() > .90:
    #         word_results = update_result(bangla_word, word_result[0], word_results)
    #         char_results = update_result(bangla_word, char_result, char_results)
    #         combined_results = update_result(bangla_word, combined, combined_results)
    #         return combined
    # else:
    #     return False

In [99]:
random.seed(0)
char_results = np.array([0, 0, 0, 0])
word_results = np.array([0, 0, 0, 0])
combined_results = np.array([0, 0, 0, 0])

start = time.time()

# results = np.array([0, 0, 0, 0])
j = 0
lines = []
for i in range(len(inp_lang)):
    outputs = predict(inp_lang[i], targ_lang[i], k=10)

    if outputs is None or outputs == False:
        continue
    # j += 1
    # lines.append(inp_lang_val[i] + ',' +targ_lang_val[i] + '\n')
    # print(outputs)
    # print(input_lang[i])
    # if targ_lang_val[i] == outputs[0]:
    #     results[0]+=1
    # if targ_lang_val[i] in outputs[0:3]:
    #     results[1]+=1
    # if targ_lang_val[i] in outputs[0:5]:
    #     results[2]+=1
    # if targ_lang_val[i] in outputs:
    #     results[3]+=1
    
    # if j % 100 == 0:
    #     # print("char", char_results / j)
    #     # print("word", word_results / j)
    #     print("combined", combined_results / j)
    #     print(f'Time taken for {i+1} examples: {(time.time() - start):.2f} s\n')
    #     with open(test_dataset_path, mode="w", encoding="utf-8") as file:
    #         file.writelines(lines)
    #     if j == 10000:
    #         break

# print(results / len(input_lang))
# print(f'Time taken: {(time.time() - start):.2f} s\n')

print("combined", combined_results / i)
print(f'Time taken for {i+1} examples: {(time.time() - start):.2f} s\n')

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_addons/seq2seq/decoder.py", line 514, in body
    next_sequence_lengths,  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/nest.py", line 870, in map_structure
    expand_composites=expand_composites)  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/nest.py", line 869, in <listcomp>
    structure[0], [func(*x) for x in entries],  File "/usr/local/lib/python3.7/dist-packages/tensorflow_addons/seq2seq/decoder.py", line 506, in <lambda>
    lambda ta, out: ta.write(time, out), outputs_ta, emit  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 249, in wrapped
    error_in_function=error_in_function)
ERROR:tensorflow:===

In [ ]:
def thread_predict(input_lang, target_lang, results, thread_id, per_thread_allocation, k=10):
    for i in range(thread_id*per_thread_allocation, 
                min((thread_id+1)*per_thread_allocation, len(input_lang)-1)):
        outputs = predict(input_lang[i], k=k)
        # print(outputs)
        # print(input_lang[i])
        if target_lang[i] == outputs[0]:
            results[thread_id][0]+=1
        if target_lang[i] in outputs[0:3]:
            results[thread_id][1]+=1
        if target_lang[i] in outputs[0:5]:
            results[thread_id][2]+=1
        if target_lang[i] in outputs:
            results[thread_id][3]+=1
        
        if i == 50:
            break


In [ ]:
# Evaluate word-level train
def calculate_acc(input_lang, target_lang):
    no_of_threads = 32
    per_thread_allocation = int(np.ceil(len(input_lang) / no_of_threads))
    threads = [None] * no_of_threads
    results = [[0, 0, 0, 0] for _ in range(no_of_threads)]
    for i in range(no_of_threads):
        x = Thread(target=thread_predict, 
                   args=(input_lang, target_lang, results, i, per_thread_allocation, ))
        x.start()
        threads[i] = x
        break
    for i in range(no_of_threads):
        threads[i].join()
        break
    # print(results)
    print(np.sum(np.array(results), axis=0))
    print(np.sum(np.array(results), axis=0) / len(input_lang))

In [ ]:
start = time.time()
calculate_acc(inp_lang_val, targ_lang_val)
print(f'Time taken: {(time.time() - start):.2f} s\n')

[12 12 14 35]
[2.49689968e-05 2.49689968e-05 2.91304963e-05 7.28262408e-05]
Time taken: 20.87 s

